#  Deep Learning
Assignment 2¶
Previously in 1_notmnist.ipynb, we created a pickle with formatted datasets for training, development and testing on the notMNIST dataset.
The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/udacity/2_fullyconnected.ipynb

In [4]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile 
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range 

Reload the data we genereted in the pickle file 

In [5]:
pickle_file='notMNIST.pickle'


with open(pickle_file , 'rb') as f:
    save=pickle.load(f)
    train_dataset=save ['train_dataset']
    train_labels=save ['train_labels']
    valid_dataset=save ['valid_dataset']
    valid_labels=save ['valid_labels']
    test_dataset=save ['test_dataset']
    test_labels=save ['test_labels']
    del save #to help compiler to free up space 
    print('train_dataset', train_dataset.shape , train_labels.shape)
    print('validation set ', valid_dataset.shape, valid_labels.shape)
    print('test set', test_dataset.shape, test_labels.shape)

train_dataset (200000, 28, 28) (200000,)
validation set  (10000, 28, 28) (10000,)
test set (10000, 28, 28) (10000,)


#  reformat into a shape that is more adapted to the models we are going to train
DATA AS FLAT MATRIX
LAB ELS AS FLOAT 1-HOT ENCODINGS

In [6]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


# Now firstly train a multinomial logistic regression (softmax regression)using simple gradient descent
how tensorflow gonna work in here :
firstly give all the inputs variable and the formula of our desired computition then these created as nodes over a computation graph 
with graph.as_default():



In [9]:
#with gradient descent training we have use small datasets as it takes longer  time
#So we should subset the train data for faster training 

train_subset=10000
graph=tf.Graph()
with graph.as_default():
    
    #Input data.
    #Load the training ,validation and data into constants that are attached to the graph
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset= tf.constant(test_dataset)
    
    #variables
    #these are the parameters that we are going to trainng .the weight matrix in inisialized using random values 
    #with normal distribution
    #the biases get initialized to zero
    
    weights=tf.Variable(
     tf.truncated_normal([image_size*image_size, num_labels]))
    #print('weights', weights)
    biases =tf.Variable(tf.zeros([num_labels]))
    #print ('biases', biases)
    
    
    #training computation
    #multiply the inputs with the weight matrix , and add biases then compute softmax and cross-entropy 
    #then take the average of this cross-entropy across all training example : that is our loss
    
    logits = tf.matmul(tf_train_dataset, weights)+ biases #scores
    loss=  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))#cross-entropy
    
    #optimizer
    #now using gradient descent for minimizing the loss after cross entropy
    
    optimizer= tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    
    #predictions for the training , validation and test data
    #these are not part of training , but merely ,here so that we can report
    #accuracy figure as we train.
    
    train_prediction=tf.nn.softmax(logits)
    valid_prediction=tf.nn.softmax(tf.matmul(tf_valid_dataset,weights) + biases)
    test_prediction= tf.nn.softmax(tf.matmul(tf_test_dataset,weights)+ biases)
    
    
    
    

now run the computation graph iteratly 

In [14]:
num_steps= 801


def accuracy(predictions, labels):
    return(100.0 * np.sum(np.argmax(predictions, 1)== np.argmax(labels, 1))/ predictions.shape[0])


with tf.Session(graph=graph) as session :
    # this is a one-time operation which ensures the parameters initialized as we described in the graph: random
    #weights for matrix and zeros for the bias 
    
    tf.global_variables_initializer().run()
    print('initialized')
    
    for step in range(num_steps):
        #run the computations ,we tell . run() that we want to run the optimizer,
        #and get the loss value and the training predictions returnes as numpy arrays
        _, l, predictions = session.run([optimizer , loss , train_prediction])
        if (step % 100 == 0):
            print('loss at step %d: %f' % (step, l))
            print('training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, : ]))
            #Calling.evaL() on valid_prediction is basically like calling run (), but just to get that one np array 
            #and it recomputes all its graph dependencies
            print('validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
         
        
        

initialized
loss at step 0: 12.443406
training accuracy: 16.8%
validation accuracy: 20.1%
loss at step 100: 2.323062
training accuracy: 71.3%
validation accuracy: 70.3%
loss at step 200: 1.906028
training accuracy: 74.3%
validation accuracy: 72.8%
loss at step 300: 1.673731
training accuracy: 75.5%
validation accuracy: 73.7%
loss at step 400: 1.510024
training accuracy: 76.5%
validation accuracy: 74.4%
loss at step 500: 1.385775
training accuracy: 77.0%
validation accuracy: 74.8%
loss at step 600: 1.286983
training accuracy: 77.6%
validation accuracy: 75.2%
loss at step 700: 1.205801
training accuracy: 78.1%
validation accuracy: 75.3%
loss at step 800: 1.137442
training accuracy: 78.5%
validation accuracy: 75.5%
test accuracy: 83.2%


# Now we are going to use Stochastic Gradient decent (SDG)
This is much faster.
The graph will be similar but this time we will not use a constant node to store the training data.
we will create a placeholder node which will be fed actual data at every call of session.run()

In [ ]:
batch_size=128
graph=tf.Graph()
with graph.as_default():
    #Input data. For the training data, we use a placeholder that will be fed 
    #at run time with a training minibatch
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Variables
    weights= tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
